## cifar10分类的卷积神经网络(模块化设计版)

### 1.导入必需模块,数据预处理

In [1]:
import os
os.chdir('/home/david/tensorflow/卷积神经网络/cifar10')   # 更改当前目录为cifar10模块下
import cifar10, cifar10_input
import tensorflow as tf
import numpy as np
import time

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# 下载数据集并解压
# cifar10.maybe_download_and_extract()  # 默认下载目录为`/tmp/cifar10_data/cifar-10-batches-bin`
batch_size = 128
data_dir = './cifar-10-batches-bin'
# 数据集预处理
images_train, labels_train = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size=batch_size)
images_test , labels_test  = cifar10_input.inputs(eval_data=True, data_dir=data_dir, batch_size=batch_size)

Filling queue with 20000 CIFAR images before starting to train. This will take a few minutes.


### 2.定义网络权重初始化函数,权重带损失

In [3]:
# 为权重添加l2损失
def variable_with_weight_loss(shape, stddev, w1, name):
    """
    功能: 为权重进行初始化,并给权重添加一定的损失
    参数: shape:权重向量的形状;stddev:标准差的大小;w1:控制权重的损失大小
    返回: 初始化的权重向量
    """
    if name is None:
        var = tf.Variable(tf.truncated_normal(shape, stddev=stddev))
    else:
        var = tf.Variable(tf.truncated_normal(shape, stddev=stddev, name=name))
    if w1 is not None:   # 为权重添加损失
        weight_loss = tf.multiply(tf.nn.l2_loss(var), w1, name='weight_loss')  # 计算weight的loss
        tf.add_to_collection('losses', weight_loss)
    return var

### 3.定义网络的函数

In [4]:
# 显示网络结构的函数
def print_activations(t):
    """
    功能: 输出网络的结构
    参数: t:输入的向量
    返回: 无
    """
    print('{:<15}{:<15}'.format(str(t.op.name), str(t.get_shape().as_list())))  # 以列表形式输出

# 网络定义的函数,2层卷积,3层全连接
def inference(images):
    """
    功能: 网络的推理函数,即网络的结构定义函数
    参数: images:输入的图像
    返回: 网络输出的特征向量,维度与分类的个数有关
    """
    # 第一层卷积层[5,5,3,64]
    with tf.name_scope('conv1') as scope:
        kernel = variable_with_weight_loss([5, 5, 3, 64], stddev=5e-2, w1=0.0, name='weight')
        conv_kernel = tf.nn.conv2d(images, kernel, [1, 1, 1, 1], padding='SAME')
        bias = tf.Variable(tf.constant(0.0, shape=[64]), name='bias')
        conv1 = tf.nn.relu(tf.nn.bias_add(conv_kernel, bias), name=scope)
        print_activations(conv1)  # 输出卷积层的结构
        pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool1')
        print_activations(pool1)  # 输出池化层的结构
        norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='norm1')
        print_activations(norm1)  # 输出lrn层的结构
    # 第二层卷积层[5, 5, 64, 64]
    with tf.name_scope('conv2') as scope:
        kernel = variable_with_weight_loss([5, 5, 64, 64], stddev=5e-2, w1=0.0, name='weight')
        conv_kernel = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        bias = tf.Variable(tf.constant(0.0, shape=[64]), name='bias')
        conv2 = tf.nn.relu(tf.nn.bias_add(conv_kernel, bias), name=scope)
        print_activations(conv2)
        norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001/9.0, beta=0.75, name='norm2')
        print_activations(norm2)
        pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1], padding='SAME', name='pool2')
        print_activations(pool2)
    # 第三层全连接层[384]
    with tf.name_scope('fc3') as scope:
        pool2_flat = tf.reshape(pool2, [batch_size, -1])   # 将feature map重塑为一维度向量
        dim = pool2_flat.get_shape()[1].value               # 获取一维向量的长度
        weight = variable_with_weight_loss([dim, 384], stddev=0.04, w1=0.004, name='weight')
        bias = tf.Variable(tf.constant(0.0, shape=[384]), name='bias')
        fc3 = tf.nn.relu(tf.matmul(pool2_flat, weight) + bias, name=scope)
        print_activations(fc3)
    # 第四层全连接层[192]
    with tf.name_scope('fc4') as scope:
        weight = variable_with_weight_loss([384, 192], stddev=0.04, w1=0.004, name='weight')
        bias = tf.Variable(tf.constant(0.0, shape=[192]), name='bias')
        fc4 = tf.nn.relu(tf.matmul(fc3, weight) + bias, name=scope)
        print_activations(fc4)
    # 第五层全连接层,输出分类层
    with tf.name_scope('logits') as scope:
        weight = variable_with_weight_loss([192, 10], stddev=1.0/192, w1=0.0, name='weight')
        bias = tf.Variable(tf.constant(0.0, shape=[10]), name='bias')
        logits = tf.add(tf.matmul(fc4, weight), bias, name=scope)         #bias_add与+,add的区别????????????
        print_activations(logits)
    return logits  # 输出网络提取的特征

### 4.定义损失函数

In [5]:
# 总的损失函数=网络输出损失+权重损失
def loss(logits, labels):
    """
    功能: 计算预测值与标签的损失+全连接层的权重损失
    参数: logits:预测输出,这里为特征不是概率; labels:训练集标签
    返回: 总的损失
    """
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=logits, \
                                                      labels=labels, name='cross_entropy_per_examples')
    print('cross_entropy:', cross_entropy)
    cross_entropy_mean = tf.reduce_mean(cross_entropy, name='cross_entropy')  # 对batch_size个数样本的损失求平均
    tf.add_to_collection('losses', cross_entropy_mean)    # 将模型输出损失添加到losses中
    losses = tf.get_collection('losses')
    print('\nlosses:', losses)
    total_loss = tf.add_n(losses, name='total_loss') # 求总的loss
    return total_loss

### 5.网络前向传播并计算损失

In [6]:
# 定义占位符变量用于传入数据
image_holder = tf.placeholder(tf.float32, [batch_size, 24, 24, 3])   
label_holder = tf.placeholder(tf.int32, [batch_size])
# 网络正向传播计算输出
logits = inference(image_holder)

conv1          [128, 24, 24, 64]
conv1/pool1    [128, 12, 12, 64]
conv1/norm1    [128, 12, 12, 64]
conv2          [128, 12, 12, 64]
conv2/norm2    [128, 12, 12, 64]
conv2/pool2    [128, 6, 6, 64]
fc3            [128, 384]     
fc4            [128, 192]     
logits         [128, 10]      


In [7]:
# 计算网络总的损失
loss = loss(logits, label_holder)
# 优化方法
train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)
# 测试网络的准确率
top_k_op = tf.nn.in_top_k(logits, label_holder, 1)  # top_k准确率,默认为1

cross_entropy: Tensor("cross_entropy_per_examples/cross_entropy_per_examples:0", shape=(128,), dtype=float32)

losses: [<tf.Tensor 'conv1/weight_loss:0' shape=() dtype=float32>, <tf.Tensor 'conv2/weight_loss:0' shape=() dtype=float32>, <tf.Tensor 'fc3/weight_loss:0' shape=() dtype=float32>, <tf.Tensor 'fc4/weight_loss:0' shape=() dtype=float32>, <tf.Tensor 'logits/weight_loss:0' shape=() dtype=float32>, <tf.Tensor 'cross_entropy:0' shape=() dtype=float32>]


### 6.创建会话,初始变量并训练网络

In [8]:
sess = tf.InteractiveSession()           # 创建会话
tf.global_variables_initializer().run()  # 全局变量初始化
tf.train.start_queue_runners()           # 开启16个线程加速图像预处理过程

[<Thread(QueueRunnerThread-input_producer-input_producer/input_producer_EnqueueMany, started daemon 140623629756160)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 140623621363456)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 140623369729792)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 140623361337088)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 140623352944384)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 140621993998080)>,
 <Thread(QueueRunnerThread-shuffle_batch/random_shuffle_queue-shuffle_batch/random_shuffle_queue_enqueue, started daemon 140621985605376)>,
 <Thread(QueueRunnerThread-shuffle_batch/

In [9]:
# 开始训练网络
for step in range(3000):
    start_time = time.time()             # 计时开始
    images_batch, labels_batch = sess.run([images_train, labels_train])  # 获取训练集
    _, loss_value = sess.run([train_op, loss], \
                             feed_dict={image_holder: images_batch, label_holder: labels_batch})
    duration = time.time() - start_time  # 获取一个batch训练的时间
    if step % 10 == 0:   # 迭代10次
        examples_per_sec = batch_size / duration  # 每s迭代的样本数
        sec_per_batch = float(duration)
        train_info = 'step %d,loss=%.2f (%.1f examples/sec; %.3f sec/batch)'
        print(train_info % (step, loss_value, examples_per_sec, sec_per_batch))

step 0,loss=4.68 (14.8 examples/sec; 8.661 sec/batch)
step 10,loss=3.63 (1824.2 examples/sec; 0.070 sec/batch)
step 20,loss=3.17 (1988.0 examples/sec; 0.064 sec/batch)
step 30,loss=2.74 (1876.7 examples/sec; 0.068 sec/batch)
step 40,loss=2.40 (1994.7 examples/sec; 0.064 sec/batch)
step 50,loss=2.42 (1982.8 examples/sec; 0.065 sec/batch)
step 60,loss=2.35 (1950.3 examples/sec; 0.066 sec/batch)
step 70,loss=2.00 (1888.1 examples/sec; 0.068 sec/batch)
step 80,loss=2.01 (1816.2 examples/sec; 0.070 sec/batch)
step 90,loss=1.82 (1825.9 examples/sec; 0.070 sec/batch)
step 100,loss=1.96 (1906.1 examples/sec; 0.067 sec/batch)
step 110,loss=2.19 (1879.0 examples/sec; 0.068 sec/batch)
step 120,loss=1.98 (1843.9 examples/sec; 0.069 sec/batch)
step 130,loss=1.83 (1927.3 examples/sec; 0.066 sec/batch)
step 140,loss=1.94 (1885.0 examples/sec; 0.068 sec/batch)
step 150,loss=1.75 (1891.7 examples/sec; 0.068 sec/batch)
step 160,loss=1.82 (2046.7 examples/sec; 0.063 sec/batch)
step 170,loss=1.78 (1966.2 

step 1410,loss=1.09 (1805.8 examples/sec; 0.071 sec/batch)
step 1420,loss=1.15 (1850.4 examples/sec; 0.069 sec/batch)
step 1430,loss=1.28 (1877.9 examples/sec; 0.068 sec/batch)
step 1440,loss=1.23 (1880.3 examples/sec; 0.068 sec/batch)
step 1450,loss=1.32 (1852.8 examples/sec; 0.069 sec/batch)
step 1460,loss=1.15 (1885.7 examples/sec; 0.068 sec/batch)
step 1470,loss=1.14 (1904.5 examples/sec; 0.067 sec/batch)
step 1480,loss=1.11 (1894.3 examples/sec; 0.068 sec/batch)
step 1490,loss=1.29 (1879.9 examples/sec; 0.068 sec/batch)
step 1500,loss=1.23 (1790.6 examples/sec; 0.071 sec/batch)
step 1510,loss=1.30 (1844.3 examples/sec; 0.069 sec/batch)
step 1520,loss=1.24 (1924.9 examples/sec; 0.066 sec/batch)
step 1530,loss=1.04 (1959.7 examples/sec; 0.065 sec/batch)
step 1540,loss=1.31 (1973.7 examples/sec; 0.065 sec/batch)
step 1550,loss=1.22 (1889.7 examples/sec; 0.068 sec/batch)
step 1560,loss=1.04 (1922.2 examples/sec; 0.067 sec/batch)
step 1570,loss=1.24 (1826.2 examples/sec; 0.070 sec/batc

step 2800,loss=0.99 (1742.6 examples/sec; 0.073 sec/batch)
step 2810,loss=1.21 (1878.7 examples/sec; 0.068 sec/batch)
step 2820,loss=1.16 (1873.8 examples/sec; 0.068 sec/batch)
step 2830,loss=0.95 (1924.8 examples/sec; 0.066 sec/batch)
step 2840,loss=0.90 (1861.2 examples/sec; 0.069 sec/batch)
step 2850,loss=0.97 (1896.9 examples/sec; 0.067 sec/batch)
step 2860,loss=0.99 (1816.2 examples/sec; 0.070 sec/batch)
step 2870,loss=1.08 (1845.3 examples/sec; 0.069 sec/batch)
step 2880,loss=1.03 (1758.8 examples/sec; 0.073 sec/batch)
step 2890,loss=0.99 (1847.8 examples/sec; 0.069 sec/batch)
step 2900,loss=0.86 (1814.6 examples/sec; 0.071 sec/batch)
step 2910,loss=1.01 (1847.6 examples/sec; 0.069 sec/batch)
step 2920,loss=1.30 (1954.7 examples/sec; 0.065 sec/batch)
step 2930,loss=1.09 (1745.9 examples/sec; 0.073 sec/batch)
step 2940,loss=0.92 (1906.3 examples/sec; 0.067 sec/batch)
step 2950,loss=1.03 (1711.7 examples/sec; 0.075 sec/batch)
step 2960,loss=1.06 (1866.9 examples/sec; 0.069 sec/batc

### 7.测试模型准确率

In [10]:
# 用测试集测试模型准确率
num_examples = 10000
import math
num_iter = int(math.ceil(num_examples / batch_size))   # ceil函数的作用是将一个小数取比它大的整数,如:30.1取31
true_count = 0
total_example_count = num_iter * batch_size
step = 0
while step < num_iter:
    image_batch, label_batch = sess.run([images_test, labels_test])   # 获取batch_size的测试数据
    # 开始测试
    predictions = sess.run([top_k_op], \
                          feed_dict={image_holder: image_batch, label_holder: label_batch})
#     print(predictions)
    print('collect number:', np.sum(predictions))  
    true_count += np.sum(predictions)              
    step += 1
    print('---------------------------------------')

collect number: 92
---------------------------------------
collect number: 92
---------------------------------------
collect number: 83
---------------------------------------
collect number: 88
---------------------------------------
collect number: 106
---------------------------------------
collect number: 82
---------------------------------------
collect number: 87
---------------------------------------
collect number: 102
---------------------------------------
collect number: 97
---------------------------------------
collect number: 96
---------------------------------------
collect number: 98
---------------------------------------
collect number: 88
---------------------------------------
collect number: 92
---------------------------------------
collect number: 88
---------------------------------------
collect number: 94
---------------------------------------
collect number: 84
---------------------------------------
collect number: 91
-----------------------------------

In [11]:
precision = true_count / total_example_count
print('precision @ 1 = %.3f' % precision)

precision @ 1 = 0.714
